## Lower-level Programming with PyTorch

Let's dig just a little deeper. We'll first get the data again.

In [28]:
from torch.autograd import Variable
import torch
import numpy as np
import matplotlib.pyplot as plt

from make_face_dataset import make_dataset


act = ['Fran Drescher', 'America Ferrera', 'Kristin Chenoweth', 'Alec Baldwin', 'Bill Hader', 'Steve Carell']
train_x, train_y = make_dataset(range(100), act)
test_x, test_y = make_dataset(range(100,120),act)

dim_x = 1024
dim_h = 20
dim_out = 6

Now,  let's define `Variable`s containing the training data

In [29]:
dtype_float = torch.FloatTensor

x = Variable(torch.from_numpy(train_x), requires_grad=False).type(dtype_float)
y = Variable(torch.from_numpy(train_y.astype(float)), requires_grad=False).type(dtype_float)

In [30]:
b0 = Variable(torch.randn((1, dim_h)), requires_grad=True)
W0 = Variable(torch.randn((dim_x, dim_h)), requires_grad=True)

b1 = Variable(torch.randn((1, dim_out)), requires_grad=True)
W1 = Variable(torch.randn((dim_h, dim_out)), requires_grad=True)

Note that everything is accessible right away:

In [31]:
b0

Variable containing:

Columns 0 to 9 
-1.1344  0.0258 -2.4930  0.3897  0.4594  0.1391 -0.5726  0.7777 -0.2557 -2.1650

Columns 10 to 19 
-0.0447  0.4612 -0.2665  0.1187  0.4125 -0.8900 -0.0408  0.5004 -1.1695 -0.2581
[torch.FloatTensor of size 1x20]

Let's now define the the model. Note that since we'll want to reuse it for different inputs, we'll want it to be in a function (or really in a class -- we'll show how to do that later). First, we'll remind ourselves of the dimensions of the data

In [32]:
print(x.data)
print(b0.data)
print(W0.data)


-0.2528 -0.2488 -0.2331  ...  -0.3116 -0.3155 -0.3116
-0.5442 -0.5795 -0.5402  ...  -0.5951 -0.4853 -0.4108
-0.5473 -0.5277 -0.5081  ...  -0.3865 -0.4101 -0.4179
          ...             ⋱             ...          
-0.2480 -0.2205 -0.2519  ...   0.3206  0.3442  0.2383
-0.4819 -0.5015 -0.3839  ...  -0.3721 -0.3957 -0.3564
-0.3708 -0.2924 -0.1943  ...   0.4606  0.4213  0.4449
[torch.FloatTensor of size 600x1024]



Columns 0 to 9 
-1.1344  0.0258 -2.4930  0.3897  0.4594  0.1391 -0.5726  0.7777 -0.2557 -2.1650

Columns 10 to 19 
-0.0447  0.4612 -0.2665  0.1187  0.4125 -0.8900 -0.0408  0.5004 -1.1695 -0.2581
[torch.FloatTensor of size 1x20]


-1.9129e-01  8.4820e-01 -1.3891e+00  ...  -3.4816e-01  5.5355e-01  1.0626e+00
 1.2477e+00 -1.1384e+00  1.2539e+00  ...   2.9954e+00 -1.4062e+00  2.7613e+00
 6.8777e-01  7.1776e-01  3.8009e-01  ...   8.4149e-01 -1.0920e+00  1.1094e+00
                ...                   ⋱                   ...                
-1.1538e+00  2.6159e+00  3.1607e+00  ..

In [33]:
print(b1.data)
print(W1.data)
print(y.data)


-0.5065 -1.2022 -0.0388 -1.1708 -2.4451  0.2631
[torch.FloatTensor of size 1x6]


 0.7867 -0.6407  1.7318 -0.3456 -0.4405  1.0894
-1.3830  0.0093  0.8043  0.9598  2.7869  0.0923
-0.3521  1.2503  0.1966 -0.3020 -0.5169  0.5635
 0.5249  1.7344 -0.9527  0.0373  0.0267 -0.9232
 1.6760 -0.6307 -0.7384  1.1678 -0.3742  0.6825
-0.8042  0.3415 -0.1001 -1.0073  0.2994  1.2488
 0.6572  0.4000 -0.9329 -0.7093  0.2007 -1.4330
-0.6654 -1.7852 -0.7719  1.4885 -0.3473 -2.7169
 1.5190 -0.2886 -0.4844  2.0773 -0.6403  1.4899
-0.2580  0.9112 -0.2918 -0.6118 -1.4890 -0.6615
 0.7023  0.1765 -0.3699  0.1038 -1.0866  0.3859
 0.2949  0.5264  2.5375  0.3050  0.6688  0.2948
-0.9225 -0.4607 -0.0293  1.2828  1.4286 -0.1895
 1.2519 -0.3709  1.5053 -1.1414 -0.4688  0.4657
-0.6303  1.0168  1.9302  0.1391 -0.5996  0.9923
 0.2467  1.6851 -1.7668  1.1029  0.9165  1.4617
-0.4769  0.6515 -0.9146 -0.0384  1.6727  0.7826
-0.2553  0.1113  1.4451  1.3871  0.5152 -1.8237
-0.5512  1.2502 -1.1458  0.7496  0.0637  0.5849
-1.04

In [34]:
def model(x, b0, W0, b1, W1):
    h = torch.matmul(x, W0) + b0.repeat(x.data.shape[0], 1)
    out = torch.matmul(h, W1) + b1.repeat(h.data.shape[0], 1)
    return out

In [35]:
y_out = model(x, b0, W0, b1, W1)

In [36]:
logSoftMax = torch.nn.LogSoftmax() # We'll be too lazy to define this one by hand
loss = -torch.mean(torch.sum(y * logSoftMax(y_out), 1))

In [37]:
loss

Variable containing:
 34.9391
[torch.FloatTensor of size 1]

In [38]:
learning_rate = 1e-1

for t in range(2000):
    y_out = model(x, b0, W0, b1, W1)
    loss = -torch.mean(torch.sum(y * logSoftMax(y_out), 1))
    loss.backward()
    b0.data -= learning_rate * b0.grad.data
    W0.data -= learning_rate * W0.grad.data
    
    b1.data -= learning_rate * b1.grad.data
    W1.data -= learning_rate * W1.grad.data
    
    
    b0.grad.data.zero_()
    W0.grad.data.zero_()
    b1.grad.data.zero_()
    W1.grad.data.zero_()
    
    if t%100 == 0:
        print("Iteration %d, Loss %.5f\n"%(t,loss.data.numpy()))

Iteration 0, Loss 34.93908

Iteration 100, Loss 1.29915

Iteration 200, Loss 0.71944

Iteration 300, Loss 0.62156

Iteration 400, Loss 0.53066

Iteration 500, Loss 0.46348

Iteration 600, Loss 0.41200

Iteration 700, Loss 0.36804

Iteration 800, Loss 0.32848

Iteration 900, Loss 0.29268

Iteration 1000, Loss 0.26009

Iteration 1100, Loss 0.23029

Iteration 1200, Loss 0.20298

Iteration 1300, Loss 0.17791

Iteration 1400, Loss 0.15487

Iteration 1500, Loss 0.13369

Iteration 1600, Loss 0.11438

Iteration 1700, Loss 0.09720

Iteration 1800, Loss 0.08260

Iteration 1900, Loss 0.07080



In [39]:
x_test_all_var = Variable(torch.from_numpy(test_x), requires_grad=False).type(dtype_float)

In [40]:
y_test_out = model(x_test_all_var, b0, W0, b1, W1).data.numpy()

In [41]:
np.argmax(y_test_out, 1)

array([0, 0, 0, 0, 0, 0, 4, 0, 1, 0, 2, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 3, 4, 2, 3, 5, 3, 5, 3, 0,
       2, 3, 3, 2, 5, 4, 3, 5, 3, 5, 5, 2, 4, 4, 2, 4, 4, 4, 4, 3, 1, 4, 4,
       0, 0, 4, 5, 4, 5, 4, 2, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 0, 5, 2, 5,
       5, 2, 2, 5, 1])

In [42]:
np.mean(np.argmax(y_test_out, 1) == np.argmax(test_y, 1))

0.70833333333333337